ETF Return Tracker
* May 17, 2025
* Version 4

Tutors:
* Anthropic's AI, Claude
* Google's AI, Gemini

This script:
* Analyzes ETF portfolio performance by calculating returns,
expenses, and net profit based on user inputs.
* Handles multiple ETFs and provide portfolio-level analysis.


Script features:
1. Prompts the user for the ETF name, expense ratio, number of shares, purchase price, purchase date, current price, and current date
2. Calculates days held between the dates
3. Computes the total return in dollars
4. Calculates the annualized return percentage
5. Estimates the total expense cost based on the expense ratio and time held
6. Determines the net return (return minus expenses)
7. Displays all results in a readable format

 I've refactored the script to include robust error handling for inputs and to allow analysis for a variable number of ETFs. I've also added some minor enhancements like ensuring dates are logical.

In [1]:
#!/usr/bin/env python3
"""
ETF Portfolio Analyzer

A Python script that analyzes a portfolio of ETFs by calculating returns,
expenses, and net profit for individual ETFs and the combined portfolio.

Features:
- Handles analysis of a user-defined number of ETFs
- Prompts the user for ETF details (name, expense ratio, shares, prices, dates)
  with input validation and error handling
- Calculates days held between purchase and current/analysis date
- Computes total return in dollars
- Calculates annualized return percentage
- Estimates total expense cost based on the expense ratio and time held
- Determines the net return (return minus expenses)
- Provides portfolio-level analysis including asset allocation percentages
- Displays results in a readable format

Version: 4
Date: May 17, 2025
Author: Created with assistance from Anthropic's Claude, Refactored by Google's Gemini
"""

import datetime

def get_validated_float_input(prompt_message, non_negative=False, strictly_positive=False):
    """Gets a validated float input from the user."""
    while True:
        try:
            value = float(input(prompt_message))
            if non_negative and value < 0:
                print("Value cannot be negative. Please try again.")
                continue
            if strictly_positive and value <= 0:
                print("Value must be greater than zero. Please try again.")
                continue
            return value
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

def get_validated_date_input(prompt_message):
    """Gets a validated date input from the user in YYYY-MM-DD format."""
    while True:
        try:
            date_str = input(prompt_message)
            # Validate format by attempting to parse
            datetime.datetime.strptime(date_str, "%Y-%m-%d")
            return date_str  # Return the string as it's used by other functions
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD.")

def calculate_days_held(start_date_str, end_date_str):
    """
    Calculate the number of days between two dates.

    Args:
        start_date_str (str): The starting date in YYYY-MM-DD format
        end_date_str (str): The ending date in YYYY-MM-DD format

    Returns:
        int: Number of days between the two dates, or 0 if dates are invalid.
    """
    try:
        start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")
        end_date = datetime.datetime.strptime(end_date_str, "%Y-%m-%d")
        if end_date < start_date: # Should be caught by input validation ideally
            return 0
        days_held = (end_date - start_date).days
        return days_held
    except ValueError:
        # This case should ideally be prevented by get_validated_date_input
        print("Error: Invalid date format encountered in calculation.")
        return 0


def calculate_annualized_return(start_price, end_price, days_held):
    """
    Calculate the annualized return percentage.

    Args:
        start_price (float): The initial price per share
        end_price (float): The current price per share
        days_held (int): Number of days the investment was held

    Returns:
        float: Annualized return as a percentage
    """
    if days_held <= 0:
        return 0.0
    if start_price <= 0: # Avoid division by zero or nonsensical returns
        return 0.0

    simple_return = (end_price - start_price) / start_price
    annualized_return = (1 + simple_return) ** (365.0 / days_held) - 1
    return annualized_return * 100  # Convert to percentage

def get_etf_data(etf_number):
    """
    Prompt the user for ETF details with validation.

    Args:
        etf_number (int): The ETF number for display purposes

    Returns:
        dict: ETF data including name, expense ratio, shares, prices, and dates
    """
    print(f"\nEnter details for ETF #{etf_number}:")
    print("-" * 30)

    etf_data = {}
    etf_data["name"] = input(f"Enter ETF #{etf_number} name (e.g., VTI, BND): ").strip()
    while not etf_data["name"]:
        print("ETF name cannot be empty.")
        etf_data["name"] = input(f"Enter ETF #{etf_number} name (e.g., VTI, BND): ").strip()

    etf_data["expense_ratio"] = get_validated_float_input(
        f"Enter expense ratio for {etf_data['name']} (as percentage, e.g., 0.05 for 0.05%): ",
        non_negative=True
    )
    etf_data["shares"] = get_validated_float_input(
        f"Enter number of shares for {etf_data['name']}: ",
        strictly_positive=True
    )
    etf_data["start_price"] = get_validated_float_input(
        f"Enter price per share on purchase date for {etf_data['name']} ($): ",
        strictly_positive=True
    )

    while True:
        etf_data["start_date"] = get_validated_date_input(
            f"Enter purchase date for {etf_data['name']} (YYYY-MM-DD): "
        )
        etf_data["end_date"] = get_validated_date_input(
            f"Enter current/analysis date for {etf_data['name']} (YYYY-MM-DD): "
        )
        try:
            start_dt = datetime.datetime.strptime(etf_data["start_date"], "%Y-%m-%d")
            end_dt = datetime.datetime.strptime(etf_data["end_date"], "%Y-%m-%d")
            if end_dt < start_dt:
                print("Error: End date cannot be before purchase date. Please re-enter dates.")
                continue
            break # Dates are valid and logical
        except ValueError: # Should not happen due to get_validated_date_input
            print("A date formatting error occurred. Please try re-entering dates.")
            continue


    etf_data["end_price"] = get_validated_float_input(
        f"Enter current/analysis price per share for {etf_data['name']} ($): ",
        non_negative=True # Price can be zero if ETF value dropped significantly
    )
    return etf_data

def analyze_etf(etf_data):
    """
    Analyze a single ETF based on provided data.

    Args:
        etf_data (dict): ETF data

    Returns:
        dict: Analysis results including returns, expenses, and performance metrics
    """
    results = {}
    results["name"] = etf_data["name"]

    expense_ratio_decimal = etf_data["expense_ratio"] / 100.0
    days_held = calculate_days_held(etf_data["start_date"], etf_data["end_date"])
    results["days_held"] = days_held

    initial_investment = etf_data["shares"] * etf_data["start_price"]
    current_value = etf_data["shares"] * etf_data["end_price"]
    total_return_dollars = current_value - initial_investment

    results["initial_investment"] = initial_investment
    results["current_value"] = current_value
    results["total_return_dollars"] = total_return_dollars

    annualized_return_pct = calculate_annualized_return(
        etf_data["start_price"], etf_data["end_price"], days_held
    )
    results["annualized_return_pct"] = annualized_return_pct

    years_held = days_held / 365.0 if days_held > 0 else 0.0
    # Expense based on initial investment, as in original script
    # A more complex model might use average value over the period
    total_expense_dollars = initial_investment * expense_ratio_decimal * years_held
    results["total_expense_dollars"] = total_expense_dollars

    net_return = total_return_dollars - total_expense_dollars
    results["net_return"] = net_return

    return results

def display_etf_results(results):
    """
    Display analysis results for a single ETF.

    Args:
        results (dict): Analysis results for the ETF
    """
    print(f"\nResults for {results['name']}:")
    print("-" * 30)
    print(f"Days Held: {results['days_held']}")
    print(f"Initial Investment: ${results['initial_investment']:.2f}")
    print(f"Current Value: ${results['current_value']:.2f}")
    print(f"Annualized Return: {results['annualized_return_pct']:.2f}%")
    print(f"Total Return: ${results['total_return_dollars']:.2f}")
    print(f"Total Expense Cost: ${results['total_expense_dollars']:.2f}")
    print(f"Return Less Expenses: ${results['net_return']:.2f}")

def analyze_portfolio(etf_results_list):
    """
    Analyze the combined portfolio of multiple ETFs.

    Args:
        etf_results_list (list): A list of analysis result dictionaries for each ETF.

    Returns:
        dict: Portfolio-level analysis including combined metrics and allocation data.
    """
    portfolio_summary = {
        "initial_investment": 0.0,
        "current_value": 0.0,
        "total_return_dollars": 0.0,
        "total_expense_dollars": 0.0,
        "net_return": 0.0,
        "weighted_annualized_return_pct": 0.0,
        "allocations": []
    }

    if not etf_results_list:
        return portfolio_summary

    for res in etf_results_list:
        portfolio_summary["initial_investment"] += res["initial_investment"]
        portfolio_summary["current_value"] += res["current_value"]
        portfolio_summary["total_return_dollars"] += res["total_return_dollars"]
        portfolio_summary["total_expense_dollars"] += res["total_expense_dollars"]
        portfolio_summary["net_return"] += res["net_return"]

    # Calculate weighted annualized return
    if portfolio_summary["initial_investment"] > 0:
        for res in etf_results_list:
            weight = res["initial_investment"] / portfolio_summary["initial_investment"]
            portfolio_summary["weighted_annualized_return_pct"] += res["annualized_return_pct"] * weight
    else:
        portfolio_summary["weighted_annualized_return_pct"] = 0.0


    # Calculate portfolio allocation percentages
    if portfolio_summary["current_value"] > 0:
        for res in etf_results_list:
            percentage = (res["current_value"] / portfolio_summary["current_value"]) * 100
            portfolio_summary["allocations"].append({
                "name": res["name"],
                "percentage": percentage
            })
    else: # Handle cases where current value is zero (e.g. all investments lost value or started at 0)
        for res in etf_results_list:
             portfolio_summary["allocations"].append({
                "name": res["name"],
                "percentage": 0.0 # Or assign based on initial investment if preferred
            })


    return portfolio_summary

def display_portfolio_results(portfolio_summary):
    """
    Display analysis results for the combined portfolio.

    Args:
        portfolio_summary (dict): Portfolio-level analysis results.
    """
    print("\nCombined Portfolio Results:")
    print("=" * 40)
    print(f"Total Initial Investment: ${portfolio_summary['initial_investment']:.2f}")
    print(f"Total Current Value: ${portfolio_summary['current_value']:.2f}")

    print("Portfolio Allocation:")
    if portfolio_summary['allocations']:
        for alloc in portfolio_summary['allocations']:
            print(f"  - {alloc['name']}: {alloc['percentage']:.1f}%")
    else:
        print("  N/A (no ETFs or zero current value)")

    print(f"Weighted Annualized Return: {portfolio_summary['weighted_annualized_return_pct']:.2f}%")
    print(f"Total Return: ${portfolio_summary['total_return_dollars']:.2f}")
    print(f"Total Expense Cost: ${portfolio_summary['total_expense_dollars']:.2f}")
    print(f"Total Net Return (Return Less Expenses): ${portfolio_summary['net_return']:.2f}")

def main():
    """
    Main function to run the ETF Portfolio Analyzer.
    """
    print("ETF Portfolio Analyzer")
    print("======================")

    while True:
        try:
            num_etfs = int(input("Enter the number of ETFs to analyze: "))
            if num_etfs <= 0:
                print("Please enter a positive number of ETFs.")
                continue
            break
        except ValueError:
            print("Invalid input. Please enter a whole number.")

    all_etf_data = []
    for i in range(num_etfs):
        all_etf_data.append(get_etf_data(i + 1))

    all_etf_results = []
    for etf_data in all_etf_data:
        all_etf_results.append(analyze_etf(etf_data))

    print("\n--- Individual ETF Analysis ---")
    for results in all_etf_results:
        display_etf_results(results)

    if all_etf_results:
        print("\n--- Combined Portfolio Analysis ---")
        portfolio_summary = analyze_portfolio(all_etf_results)
        display_portfolio_results(portfolio_summary)
    else:
        print("\nNo ETF data to analyze for portfolio summary.")

    print("\nAnalysis complete.")

if __name__ == "__main__":
    main()

ETF Portfolio Analyzer
Enter the number of ETFs to analyze: 3

Enter details for ETF #1:
------------------------------
Enter ETF #1 name (e.g., VTI, BND): VTI
Enter expense ratio for VTI (as percentage, e.g., 0.05 for 0.05%): 0.03
Enter number of shares for VTI: 10
Enter price per share on purchase date for VTI ($): 206.55
Enter purchase date for VTI (YYYY-MM-DD): 2022-05-31
Enter current/analysis date for VTI (YYYY-MM-DD): 2025-05017
Invalid date format. Please use YYYY-MM-DD.
Enter current/analysis date for VTI (YYYY-MM-DD): BND
Invalid date format. Please use YYYY-MM-DD.
Enter current/analysis date for VTI (YYYY-MM-DD): 2025-05-17
Enter current/analysis price per share for VTI ($): 292.58

Enter details for ETF #2:
------------------------------
Enter ETF #2 name (e.g., VTI, BND): bnd
Enter expense ratio for bnd (as percentage, e.g., 0.05 for 0.05%): 0.03
Enter number of shares for bnd: 3
Enter price per share on purchase date for bnd ($): 76.55
Enter purchase date for bnd (YYYY-MM